# **Modélisation des distributions d’espèces** <br>*Les effets de priorité et la limitation de la dispersion affectent-ils les possibilités d’inférence des déterminants de la niche ?*
*Présentation du 21 mars 2023*


In [ ]:
# jupyter nbconvert Presentation.ipynb --no-input --to html

## **Introduction**

### 1. *La niche écologique*

Imaginons une espèce quelconque, et un paysage avec plusieurs sites qui présentent des caractéristiques différentes. La capacité de cette espèce à survivre dans un site en particulier est déterminée par sa réponse aux caractéristiques de ce site. Par exemple, est-ce que l'exposition au soleil du site A est suffisante pour permettre à une certaine espèce de plante de se développer ? Est-ce que la température en hiver du site B est assez haute pour permettre à une certaine espèce de reptile de survivre ? L'ensemble des valeurs des caractéristiques qui permettent à l'espèce de subsister dans un milieu est la niche de cette espèce. 

<div class="alert alert-block alert-info">
La niche fondamentale d’une espèce est définie comme un hypervolume à n dimensions, où chaque dimension représente une variable environnementale qui a un impact sur la survie de l’espèce (Hutchinson, 1957). Les effets de ces variables sont ce qu'on appelle les <b>déterminants de la niche</b>.

<figure>
<center><img src="Niche2D.png" style="width:30%" /></center>
<figcaption align = "center"> <b>Figure 1.</b> Diagramme BAM (Biotique Abiotique Mouvement). <b>A</b> correspond à l’ensemble des zones où les conditions abiotiques sont favorables à la présence de l’espèce et correspond à la niche fondamentale selon Hutchinson (<b>NF</b>). <b>B</b> correspond à l’ensemble des zones où les conditions biotiques sont favorables à la présence de l’espèce. <b>M</b> correspond à la capacité de l’espèce de disperser vers un site à partir d’une zone occupée. L’intersection entre <b>A</b> et <b>B</b> correspond à la niche réalisée selon Hutchinson (<b>NR</b>). L’intersection <b>P</b> entre <b>A</b>, <b>B</b> et <b>M</b> est la distribution réelle de l’espèce. </figcaption>
</figure>

</div>



<figure>
<center><img src="Hutchinson.png" style="width:30%" /></center>
<figcaption align = "center"> <b>Figure 1.</b> Diagramme BAM (Biotique Abiotique Mouvement). <b>A</b> correspond à l’ensemble des zones où les conditions abiotiques sont favorables à la présence de l’espèce et correspond à la niche fondamentale selon Hutchinson (<b>NF</b>). <b>B</b> correspond à l’ensemble des zones où les conditions biotiques sont favorables à la présence de l’espèce. <b>M</b> correspond à la capacité de l’espèce de disperser vers un site à partir d’une zone occupée. L’intersection entre <b>A</b> et <b>B</b> correspond à la niche réalisée selon Hutchinson (<b>NR</b>). L’intersection <b>P</b> entre <b>A</b>, <b>B</b> et <b>M</b> est la distribution réelle de l’espèce. </figcaption>
</figure>

### 2. *Les modèles de distribution d'espèces*

### 3. *Effets issus de l'écologie des communautés*

## I. Simulation d'une méta-communauté

<div class="alert alert-block alert-info">
<b>Nomenclature:</b> Pour la suite de cette partie, nous considérons <b>2 espèces</b> qui se partagent <b>2 ressources</b>.<br>L'<b><font color='blue'>espèce 1</font></b>, généraliste, peut utiliser la <b><font color='green'>ressource 1</font></b> et la <b><font color='red'>ressource 2</font></b> de façon équivalente.<br>L'<b><font color='orange'>espèce 2</font></b> est spécialiste de la <b><font color='red'>ressource 2</font></b>.
</div>

$$(e_{i1})^{1/s}+(e_{i2})^{1/s}=1$$

## II. Les modèles de distribution conjointe des espèces (JSDM)